In [1]:
# @title init for running on colab
!pip install datasets
!pip install tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.5 MB/s eta 0:00:00


In [2]:
# @title load data source


DATASET_SOURCE = 'local_file' # @param ["wikimedia/wikipedia/20231101.zh-classical", "local_file"]

LOCAL_FILE = 'input.txt' # @param {type:"string"}

print(f'{DATASET_SOURCE=}')

def load_dataset_from_source():
    if DATASET_SOURCE == 'local_file':
        with open(LOCAL_FILE, 'r', encoding='utf-8') as f:
            text = f.read()
        print('final input text length', len(text))
        return text

    text = ''
    if DATASET_SOURCE == 'wikimedia/wikipedia/20231101.zh-classical':
        dataset = load_dataset('wikimedia/wikipedia', '20231101.zh-classical')
        num_rows = dataset['train'].num_rows

    for i in range(num_rows):
        text += dataset['train'][i]['text']
    print('final input text length', len(text))
    return text


INPUT_TEXT = load_dataset_from_source()

DATASET_SOURCE='local_file'
final input text length 1115394


In [3]:
# @title select tokenization alg

TOKENIZATION_ALG = 'bpe'  # @param ["default", "tiny-llama-fast-tokenizer", "bpe"]
TRAINING_DATA_SPLIT = 0.9  # @param {type:"slider", min:0, max:1, step:0.05}

# @markdown The following hyper parameters are only applicable for BPE
BPE_VOCAB_SIZE = 276 # @param {type:"number"}

print(f'{TOKENIZATION_ALG=} {TRAINING_DATA_SPLIT=}')


def bpe():

    def get_byte_pair_stats(tokens):
        """
        Return one byte pair with the highest frequency
        """
        counts = {}
        for pair in zip(tokens, tokens[1:]): # iterate through consecutive elements
            counts[pair] = counts.get(pair, 0) + 1
        return counts


    def merge_byte_pair(tokens, pair, new_token):
        """
        tokens: list of token bytes
        pair: byte pair to be replaced
        new_token: new byte

        return a new list of token bytes after replacement

        example: merge_byte_pair([5,6,6,7,9], (6,7), 99) -> [5, 6, 99, 9]
        """
        i = 0
        new_tokens = []
        while i < len(tokens):
            if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i+1] == pair[1]:
                new_tokens.append(new_token)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        return new_tokens

    print("begin byte pair tokenization")
    input_text_tokens = INPUT_TEXT.encode("utf-8") # utf-8 encoded byte array
    input_text_tokens = list(map(int, input_text_tokens)) # convert all bytes into int
    original_tokens_len = len(input_text_tokens)

    # develop the merge forest from the input data
    num_merges = BPE_VOCAB_SIZE - 256
    merges = {} # (int, int) -> int
    for i in range(num_merges):
        all_byte_pair = get_byte_pair_stats(tokens=input_text_tokens)
        top_byte_pair = max(all_byte_pair, key=all_byte_pair.get)
        new_token = 256 + i
        print(f"merging {top_byte_pair} into a new token {new_token}")
        input_text_tokens = merge_byte_pair(tokens=input_text_tokens, pair=top_byte_pair, new_token=new_token)
        merges[top_byte_pair] = new_token

    print(f"original token length {original_tokens_len}; now token length {len(input_text_tokens)}; compression %: {original_tokens_len / len(input_text_tokens):.2f}X")

    # calculate new vocab set
    vocab = {idx: bytes([idx]) for idx in range(256)}
    for (p0, p1), idx in merges.items():
        vocab[idx] = vocab[p0] + vocab[p1]

    def bpe_decode(input_tokens):
        unmerged_tokens = b"".join(vocab[i] for i in input_tokens)
        return unmerged_tokens.decode("utf-8", errors="replace")

    def bpe_encode(text):
        tokens = list(text.encode("utf-8"))
        while len(tokens) >= 2:
            all_byte_pairs = get_byte_pair_stats(tokens=tokens)
            pair = min(all_byte_pairs, key=lambda p: merges.get(p, float("inf")))
            if pair not in merges:
                break # the pair is not from the merge forest
            new_token = merges[pair]
            tokens = merge_byte_pair(tokens=tokens, pair=pair, new_token=new_token)
        return tokens

    all_data = torch.tensor(input_text_tokens, dtype=torch.long)

    return all_data, bpe_decode, bpe_encode



def tokenize(tokenization_alg):
    """
    input: type, one of "default", "tiny-llama-fast-tokenizer"
    output (training_data, validation_data, decode, encode, vocab_size)
      decode, encode are functions
    """

    if tokenization_alg == 'default':
        # each token is one character
        all_distinct_characters = sorted(list(set(INPUT_TEXT)))
        vocab_size = len(all_distinct_characters)
        stoi = {ch: i for i, ch in enumerate(all_distinct_characters)}
        itos = {i: ch for i, ch in enumerate(all_distinct_characters)}
        # encoder: take a string, output a list of integers
        def encode(s): return [stoi[c] for c in s]
        # decoder: take a list of integers, output a string
        def decode(l): return ''.join([itos[i] for i in l])

        # encode text to long
        all_data = torch.tensor(encode(INPUT_TEXT), dtype=torch.long)

    elif tokenization_alg == 'bpe':
        all_data, decode, encode = bpe()
        vocab_size = BPE_VOCAB_SIZE

    elif tokenization_alg == 'tiny-llama-fast-tokenizer':
        vocab_size = 32000
        tokenizer = AutoTokenizer.from_pretrained(
            'fxmarty/tiny-llama-fast-tokenizer')
        all_data = torch.tensor(tokenizer(INPUT_TEXT).input_ids, dtype=torch.long)
        decode = tokenizer.decode
        encode = tokenizer.encode

    train_data_size = int(TRAINING_DATA_SPLIT * len(all_data))
    train_data = all_data[:train_data_size]
    validation_data = all_data[train_data_size:]

    print(f'{vocab_size=}')
    return train_data, validation_data, decode, encode, vocab_size


train_data, validation_data, decode, encode, vocab_size = tokenize(TOKENIZATION_ALG)

TOKENIZATION_ALG='bpe' TRAINING_DATA_SPLIT=0.9
begin byte pair tokenization
merging (101, 32) into a new token 256
merging (116, 104) into a new token 257
merging (116, 32) into a new token 258
merging (115, 32) into a new token 259
merging (100, 32) into a new token 260
merging (44, 32) into a new token 261
merging (111, 117) into a new token 262
merging (101, 114) into a new token 263
merging (105, 110) into a new token 264
merging (121, 32) into a new token 265
merging (97, 110) into a new token 266
merging (58, 10) into a new token 267
merging (111, 114) into a new token 268
merging (111, 32) into a new token 269
merging (101, 110) into a new token 270
merging (10, 10) into a new token 271
merging (97, 114) into a new token 272
merging (32, 257) into a new token 273
merging (111, 110) into a new token 274
merging (108, 108) into a new token 275
original token length 1115394; now token length 882737; compression %: 1.26X
vocab_size=276


In [8]:
# @title initialize model hyper parameter

TORCH_MANUAL_SEED = 1337  # @param {type:"number"}
print(f'{TORCH_MANUAL_SEED=}')
torch.manual_seed(TORCH_MANUAL_SEED)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'{DEVICE=}')

# @markdown how many independent sequences will we process in parallel
BATCH_SIZE = 32  # @param {type:"number"}
# @markdown what's the maximum context length for prediction
BLOCK_SIZE = 128  # @param {type:"number"}
print(f'{BATCH_SIZE=} {BLOCK_SIZE=}')

MAX_TRAINING_ITERATIONS = 5000  # @param {type:"number"}
LEARNING_RATE = 3e-4  # @param {type:"number"}
print(f'{MAX_TRAINING_ITERATIONS=} {LEARNING_RATE=}')

# @markdown for every EVAL_INTERVAL training iterations, we run evaluation
EVAL_INTERVAL = 1000  # @param {type:"number"}
# @markdown for each evaluation we run # EVAL_ITERS of iterations
EVAL_ITERS = 20  # @param {type:"number"}
print(f'{EVAL_INTERVAL=} {EVAL_ITERS=}')

# @markdown number of embedding dimension
N_EMBED = 100  # @param {type:"number"}
# @markdown number of heads in multihead; N_EMBED needs to be divisible by N_HEAD
N_HEAD = 5  # @param {type:"number"}
# @markdown number of layers to run sequentially
N_LAYER = 6  # @param {type:"number"}
print(f'{N_EMBED=} {N_HEAD=} {N_LAYER=}')

# @markdown drop out rate for regularization
DROP_OUT = 0.2  # @param {type:"number"}
print(f'{DROP_OUT=}')

TORCH_MANUAL_SEED=1337
DEVICE='cuda'
BATCH_SIZE=32 BLOCK_SIZE=128
MAX_TRAINING_ITERATIONS=5000 LEARNING_RATE=0.0003
EVAL_INTERVAL=1000 EVAL_ITERS=20
N_EMBED=100 N_HEAD=5 N_LAYER=6
DROP_OUT=0.2


In [9]:
# @title Transformer!

# TODO: rewrite get_batch function so it's not random
# we should run epoch
def get_batch(split):
    """
    generate a small batch (of BATCH_SIZE) of data of input x and target y

    input:
        split: either "train" or "eval"
    output:
        (x, y): both x and y are of the same shape (BATCH_SIZE, BLOCK_SIZE);
        y is 1 right shift from x
    """
    data = train_data if split == 'train' else validation_data

    # len(data) - BLOCK_SIZE is the maximum random int
    # - BLOCK_SIZE because the last BLOCK_SIZE, the response variable y will go out of boundary
    # (BATCH_SIZE, ) is one dimensional array of size BATCH_SIZE
    random_idx = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE, ))

    # x and y are (BATCH_SIZE, BLOCK_SIZE) shape
    # y is right shift by 1 from x
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in random_idx])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in random_idx])
    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y


@torch.no_grad
def evaluate_loss():
    out = {}
    model.eval()  # setting to evaluation state
    for split in ['train', 'eval']:
        losses = torch.zeros(EVAL_ITERS)
        # for each EVAL_INTERVAL steps in training, we EVAL_ITERS steps to estimate loss
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()  # setting to training state
    return out


class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(N_EMBED, head_size, bias=False)
        self.query = nn.Linear(N_EMBED, head_size, bias=False)
        self.value = nn.Linear(N_EMBED, head_size, bias=False)
        self.register_buffer('tril', torch.tril(
            torch.ones(BLOCK_SIZE, BLOCK_SIZE)))
        self.dropout = nn.Dropout(DROP_OUT)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # key (B, T, C)
        q = self.query(x)  # query
        # (B T C) @ (B C T) -> (B T T); divide by sqrt(d_k) to preserve variance
        weight = q @ k.transpose(-2, -1) * C**-0.5
        weight = weight.masked_fill(self.tril[:T, :T] == 0, float(
            '-inf'))  # mark all up right triangle and -inf
        weight = F.softmax(weight, dim=-1)  # B T T
        weight = self.dropout(weight)
        # weighted aggregation of the values
        value = self.value(x)
        return weight @ value


class MultiHead(nn.Module):
    """multiple heads of self attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(N_EMBED, N_EMBED)
        self.dropout = nn.Dropout(DROP_OUT)

    def forward(self, x):
        # concat over the channel dimension
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            # from pager section 3.3, d_ff = d_model * 4
            nn.Linear(N_EMBED, 4 * N_EMBED),
            nn.ReLU(),
            nn.Linear(4 * N_EMBED, N_EMBED),  # residual connection
            nn.Dropout(DROP_OUT),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self):
        super().__init__()
        head_size = N_EMBED // N_HEAD
        self.sa = MultiHead(N_HEAD, head_size)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(N_EMBED)  # layer norm
        self.ln2 = nn.LayerNorm(N_EMBED)

    def forward(self, x):
        # x = x + self.sa(self.ln1(x))  # residual connection
        x = self.ln1(x + self.sa(x))
        # x = x + self.ffwd(self.ln2(x))
        x = self.ln2(x + self.ffwd(x))
        return x


class BigramLM(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=N_EMBED)
        self.position_embedding_table = nn.Embedding(
            num_embeddings=BLOCK_SIZE, embedding_dim=N_EMBED)
        # language model head
        # a linear transformation
        self.blocks = nn.Sequential(
            *[Block() for _ in range(N_LAYER)]
        )
        self.ln_f = nn.LayerNorm(N_EMBED)  # final layer norm
        self.lm_head = nn.Linear(in_features=N_EMBED, out_features=vocab_size)

    def forward(self, x_batch, target=None):
        """
        input:
            x_batch, target: (BATCH_SIZE, time) shape tensor
                BATCH_SIZE = 4; time = 8 = BLOCK_SIZE
        return:
            logits: shape (B * T, C)
            loss: optional float
        """

        B, T = x_batch.shape

        # logits of shape (BATCH_SIZE, time, channel)
        # channel is vocab size
        # for each batch, we predict for each index, the logits to predict the next char
        # among all vocab size
        token_embed = self.token_embedding_table(x_batch)  # (B, T, n_embed)
        position_embed = self.position_embedding_table(
            torch.arange(T, device=DEVICE))  # shape (T, C)
        x = token_embed + position_embed
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        # loss function = negative log likelihood (cross entropy against targets)
        # need to reshape logits and y_batch for the cross_entropy function
        if target is None:
            return logits, None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target=target)
            return logits, loss

    def generate(self, x_batch, max_new_tokens):
        """
        generate the next max_new_tokens from x_batch, being the current context

        input:
            x_batch: (B, T) array of indices
        """
        # TODO: we should update the x batch with the newly generated token
        for _ in range(max_new_tokens):
            # crop out the last block size
            x_batch_bounded = x_batch[:, -BLOCK_SIZE:]
            logits, loss = self(x_batch_bounded)
            # becomes (B, C), get the last column in T
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            predicted_next_index = torch.multinomial(
                probs, num_samples=1)  # (B, 1)
            x_batch = torch.cat(
                (x_batch, predicted_next_index), dim=1)  # (B, T+1)
        return x_batch


model = BigramLM()
m = model.to(device=DEVICE)
print('Model has', sum(p.numel() for p in m.parameters()) / 1e6, "M param")

# predict 100 tokens from token value 0
print("\nGenerate 100 tokens from untrained model:\n")
print(decode(m.generate(x_batch=torch.zeros(
    (1, 1), dtype=torch.long, device=DEVICE), max_new_tokens=100)[0].tolist()))
print('\n')


# create pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=LEARNING_RATE)

# train for these many iterations
print(f"Begin training for {MAX_TRAINING_ITERATIONS} iterations")
for step in tqdm(range(MAX_TRAINING_ITERATIONS)):
    if step % EVAL_INTERVAL == 0:
        losses = evaluate_loss()
        print(
            f"Evaluation: train loss {losses['train']:.4f}, val loss {losses['eval']:.4f}")

    x_batch, y_batch = get_batch('train')
    logits, loss = m(x_batch, y_batch)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f'Training Completed')
print(f"Loss after training {loss.item()}")

# re try generate tokens after training
print("Generate 100 tokens from trained model with empty context")
empty_context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
print(decode(m.generate(x_batch=empty_context,
      max_new_tokens=100)[0].tolist()))

Model has 0.794476 M param

Generate 100 tokens from untrained model:

�<ܙƓ��2����ˬ�H�䰪Eo �A�}M�ۻ�]:�I��=������C<����#s g Pll�[��


Begin training for 5000 iterations


  0%|          | 1/5000 [00:00<55:13,  1.51it/s]

Evaluation: train loss 5.8524, val loss 5.8492


 20%|██        | 1004/5000 [00:44<05:04, 13.14it/s]

Evaluation: train loss 2.6707, val loss 2.7224


...

Evaluation: train loss 2.2036, val loss 2.3757


 80%|████████  | 4003/5000 [03:00<01:30, 10.97it/s]

Evaluation: train loss 2.0993, val loss 2.3015


100%|██████████| 5000/5000 [03:44<00:00, 22.28it/s]


Training Completed
Loss after training 2.1685941219329834
Generate 100 tokens from trained model with empty context
 urse
As sucuemen me jothersed and Anoidght;
From for swerd with weed. Warwif worls,
To be with out it while the cromed tend a


In [ ]:
CONTEXT = ' '  # @param {type:"string"}
print(f'{CONTEXT=}')
GENERATE_TOKEN_LIMIT = 1000  # @param {type:"number"}
print(f'{GENERATE_TOKEN_LIMIT=}')

encoded_context = torch.tensor(
    [encode(CONTEXT)], dtype=torch.long, device=DEVICE)

print('\nGenerate from model:\n')
print(decode(m.generate(x_batch=encoded_context,
      max_new_tokens=GENERATE_TOKEN_LIMIT)[0].tolist()))

CONTEXT=' '
GENERATE_TOKEN_LIMIT=1000

Generate from model:

